In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import date
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

In [3]:
class MovingAverageBacktester:
    def __init__(self, data, min_fast_ma, max_fast_ma, 
                 min_slow_ma, max_slow_ma, assetData,
                 sign = 1, maType = "simple", minReturn = 0.12, maxRebalance = 1000,
                 baseAsset = "Largecap", secondAsset = "Gold", name = "MA"):
        ## Raw Data
        self.data = data
        ## Min and Max values for FAST and SLOW moving averages
        self.min_fast_ma = min_fast_ma
        self.max_fast_ma = max_fast_ma
        self.min_slow_ma = min_slow_ma
        self.max_slow_ma = max_slow_ma
        self.sign = sign
        
        ## MIN CAGR requirement and MAX rebalances
        self.minReturn = minReturn
        self.maxRebalance = maxRebalance
        
        ## Asset Class NAV Data
        self.assetData = assetData
        self.maType = maType
        
        ## Base Asset and Second Asset Name
        self.baseAsset = baseAsset
        self.secondAsset = secondAsset
        ## Strategy Name or Raw Data name
        self.name = name
        
        ## Empty Dataframe to store the results
        self.results = pd.DataFrame(columns=['Fast_MA', 'Slow_MA', 'Total_Return', 'Num_Rebalances'])
        ## Empty Dataframe to store the NAV of each combos
        self.NAV = pd.DataFrame()
        ## Empty Dataframe to store the signal for each combos
        self.maSignal = pd.DataFrame()


    def run_backtest(self):
        
        ## Calculate the total number of iteration or number of combinations
        total_iterations = (self.max_fast_ma - self.min_fast_ma + 1) * (self.max_slow_ma - self.min_slow_ma + 1)
        
        ## Progress BAR
        with tqdm(total=total_iterations, desc="Backtesting") as pbar:
            
            ## Nested For loop for multiple MA ocmbinations
            for fast_ma in range(self.min_fast_ma, self.max_fast_ma + 1):
                for slow_ma in range(self.min_slow_ma, self.max_slow_ma + 1):
                    
                    ## Execute only is FAST_MA is less than SLOW_MA
                    if fast_ma < slow_ma:
                        
                        ## Function Call for signal generation
                        signals = self.generate_signals(fast_ma, slow_ma)
                        ## Function Call for return calculation
                        total_return = self.calculate_total_return(signals, fast_ma, slow_ma)
                        ## Calculate the number of signal switches
                        num_rebalances = len(signals[signals['Position'] != 0])
                        
                        ## Store the result in a variable and append to master result dataframe
                        result = pd.DataFrame({'Fast_MA': [fast_ma],
                                               'Slow_MA': [slow_ma],
                                               'Total_Return': [total_return],
                                               'Num_Rebalances': [num_rebalances]})
                        self.results = pd.concat([self.results, result])
                        
                        ## Store the signal for all combos in master dataframe
                        comboName = f"{self.name}_Fast({fast_ma})_Slow({slow_ma})"
                        self.maSignal = pd.concat([self.maSignal, 
                                                   signals["Signal"].rename(comboName)], axis = 1)
                        self.maSignal.index = pd.to_datetime(self.maSignal.index)

                    ## Update the Progress BAR  
                    pbar.update(1)
                    
        ## Filter the final result as per Min CAGR Requirements and MAX Rebalances.
        cond1 = self.results["Num_Rebalances"]<=self.maxRebalance
        cond2 = self.results["Total_Return"] >= self.minReturn
        self.results = self.results[cond1 & cond2]
        self.results = self.results.sort_values("Total_Return", ascending = False).reset_index(drop = True)
        
        ## Function call to generate Signal charts
        self.generateSignalReport()
                                                       

    def generate_signals(self, fast_ma, slow_ma):
        ## Empty Dataframe to store the signal
        signals = pd.DataFrame(index=self.data.index)
        ## Rolling Moving Average for FAST and SLOW period
        if self.maType == "simple":
            signals['Fast_MA'] = self.data['Close'].rolling(window=fast_ma, min_periods=1).mean()
            signals['Slow_MA'] = self.data['Close'].rolling(window=slow_ma, min_periods=1).mean()
        elif self.maType == "ema":
            signals['Fast_MA'] = self.data['Close'].ewm(fast_ma, min_periods = 1).mean()
            signals['Slow_MA'] = self.data['Close'].ewm(slow_ma, min_periods = 1).mean()
            
        
        ## Signal Condition
        signals['Signal'] = 0.0
        # condition = signals["Fast_MA"] <= signals["Slow_MA"]
        condition = ((signals["Fast_MA"]-signals["Slow_MA"])*self.sign)>=0
        
        ## Generate the Signal Columns
        signals['Signal'] = np.where(condition, 1, 0)
        
        ## Asset Weight for asset classes
        signals["BaseAsset"] = np.where(condition, 1, 0)
        signals["SecondAsset"] = np.where(condition, 0, 1)
        signals['Position'] = signals['Signal'].diff()
        
        return signals
    
    def calculate_total_return(self, signals, fast_ma, slow_ma):
        
        ## Filter the data for required asset classes
        assetData = self.assetData.filter(items = [self.baseAsset, self.secondAsset])
        assetData = assetData.loc[min(signals.index):]
        
        ## MErge the price data and Singal
        data = pd.merge_asof(assetData, signals, left_index = True, right_index = True)

        price = data.filter(items = [self.baseAsset, self.secondAsset])

        sig = data.filter(items = ["BaseAsset", "SecondAsset"])
        sig.columns = [self.baseAsset, self.secondAsset]
        
        ## Percentage returns and Portfolio performance
        assetRtrn = price.pct_change()
        assetRtrn['Portfolio'] = (assetRtrn*sig.shift(1)).sum(axis = 1)
        
        ## Calculate the NAV
        port = assetRtrn.add(1).cumprod().fillna(1)*100

        ## CAGR
        dur = (port.index[-1]-port.index[0]).days/365
        port_cagr = (port['Portfolio'][-1]/port['Portfolio'][0])**(1/dur)-1
        
        ## Store the NAV in master dataframe
        strategy_nav = port["Portfolio"].copy()
        strategy_nav = strategy_nav.rename(f"{self.name}_Fast({fast_ma})_Slow({slow_ma})")
        self.NAV = pd.concat([self.NAV, strategy_nav], axis = 1)
        
        return port_cagr
    
    def generateSignalReport(self,):
        
        if len(self.results) <= 0:
            print("There are no good results as per given input values. May be try with different values.")
            return
        
        ## Color map for signal
        color_map = {1 : 'green', 0 : 'red'}
        
        ## Create the PDF object
        signalPDF = PdfPages(f'{self.name}_Signal_Report_{self.maType.title()}-MovingAverage_{date.today()}.pdf')

        #######################
        ## Looping for Top results
        for idx, row in self.results.head(5).iterrows():
            
            ## Strategy Name
            strategyName = f'{self.name}_Fast({row["Fast_MA"]})_Slow({row["Slow_MA"]})'

            ## Plot Object
            fig, ax = plt.subplots(figsize=(20,10), dpi = 300)
            
            ## Generate Background color as Green | Red for Buy | Sell respectively.
            for zn in self.maSignal[strategyName].dropna().unique():
                ax.fill_between(self.maSignal[strategyName].reset_index()["index"],
                                0,1.5,
                                where = self.maSignal[strategyName] == zn,
                                alpha = .4,
                                color = color_map[zn],
                                label = zn, interpolate = True,
                                transform = ax.get_xaxis_transform())
            plt.yticks([])
            ## Plot the Largecap Data
            plt.plot(self.assetData["Largecap"], label = 'Nifty')
            plt.legend()
    
            ## CAGR | Rebalance Title Text.
            cagr= f"{round(row['Total_Return']*100,2)}-CAGR"
            rebal = f"{row['Num_Rebalances']}-Switches"   
            ## Title
            plt.suptitle(f"{self.name} -  {cagr} | {rebal}", fontsize = 20)
            plt.title(f"Method: Moving Average Crossover  |  Fast-MA: {row['Fast_MA']} |  Slow-MA: {row['Slow_MA']}")
            
            ## Legend
            plt.legend(loc = "lower left", fontsize = 15)
            plt.tight_layout()
            
            ## Save the fig to PDF
            signalPDF.savefig(fig)
            ## Close the close
            plt.close()
            
        ## Close the PDF file
        signalPDF.close()

In [4]:
class BollingerBandBacktester:
    def __init__(self, data, min_ma, max_ma,min_thresh, max_thresh,
                 assetData, sign = 1, minReturn = 0.12, maxRebalance = 1000,
                 baseAsset = "Largecap", secondAsset = "Gold", name = "MA"):
        ## Raw Data
        self.data = data
        ## Min and Max values for FAST and SLOW moving averages
        self.min_ma = min_ma
        self.max_ma = max_ma
        self.min_thresh = min_thresh
        self.max_thresh = max_thresh
 
        ## MIN CAGR requirement and MAX rebalances
        self.minReturn = minReturn
        self.maxRebalance = maxRebalance
        
        ## Asset Class NAV Data
        self.assetData = assetData
        self.sign = sign
        
        ## Base Asset and Second Asset Name
        self.baseAsset = baseAsset
        self.secondAsset = secondAsset
        ## Strategy Name or Raw Data name
        self.name = name
        
        ## Empty Dataframe to store the results
        self.results = pd.DataFrame(columns=['MA', 'Threshold',"EntryType", "ExitType",
                                             'Total_Return', 'Num_Rebalances'])
        ## Empty Dataframe to store the NAV of each combos
        self.NAV = pd.DataFrame()
        ## Empty Dataframe to store the signal for each combos
        self.maSignal = pd.DataFrame()
        
        
    def run_backtest(self):

        total_iterations = len(range(self.min_ma, self.max_ma + 1))*4*len(np.arange(self.min_thresh, self.max_thresh+0.1, 0.1))
        ## Progress BAR
        with tqdm(total=total_iterations, desc="Backtesting") as pbar:
            
            ## Looping for two type of entry signal
            for entryType in ["enter", "exit"]:
                ## Looping for two type of exit signal
                for exitType in ["enter", "exit"]:
                    ## Looping for multiple MA values
                    for ma in range(self.min_ma, self.max_ma + 1):
                        ## Looping for multiple thresholds
                        for thresh in np.arange(self.min_thresh, self.max_thresh+0.1, 0.1):
                        
                            ## Function Call for signal generation
                            signals = self.generate_signals(ma,thresh,entryType,exitType)
                            ## Function Call for return calculation
                            total_return = self.calculate_total_return(signals,ma,thresh,entryType,exitType)
                            ## Calculate the number of signal switches
                            num_rebalances = len(signals[signals['Position'] != 0])
                            
                            ## Store the result in a variable and append to master result dataframe
                            result = pd.DataFrame({'MA': [ma],
                                                   'Threshold': [thresh],
                                                   "EntryType" : [entryType],
                                                   "ExitType" : [exitType],
                                                   'Total_Return': [total_return],
                                                   'Num_Rebalances': [num_rebalances]})
                            self.results = pd.concat([self.results, result])
                            
                            ## Store the signal for all combos in master dataframe
                            comboName = f"{self.name}_MA({ma})_Thresh({thresh})_Entry({entryType})_Exit({exitType})"
                            self.maSignal = pd.concat([self.maSignal, 
                                                       signals["Signal"].rename(comboName)], axis = 1)
                            self.maSignal.index = pd.to_datetime(self.maSignal.index)
                        
                            pbar.update(1)
                            
        ## Filter the final result as per Min CAGR Requirements and MAX Rebalances.
        cond1 = self.results["Num_Rebalances"]<=self.maxRebalance
        cond2 = self.results["Total_Return"] >= self.minReturn
        self.results = self.results[cond1 & cond2]
        self.results = self.results.sort_values("Total_Return", ascending = False).reset_index(drop = True)
        
        ## Function call to generate Signal charts
        self.generateSignalReport()
 
                      
    def generate_signals(self, ma, thresh, entryType, exitType):
        
        ## Empty Dataframe to store the signal
        signals = pd.DataFrame(index=self.data.index)
        signals["Value"] = self.data["Close"].copy()
        
        ## Rolling and Std.Deviation
        signals["Mean"] = signals["Value"].rolling(ma).mean()
        signals["Std"] = signals["Value"].rolling(ma).std()
        ## Upper and Lower Band calculation
        signals["UB"] = signals["Mean"] + thresh * signals["Std"]
        signals["LB"] = signals["Mean"] - thresh * signals["Std"]
        ## Oscillator of LB and UB
        signals["LB_Delta"] = signals["Value"] - signals["LB"]
        signals["UB_Delta"] = signals["Value"] - signals["UB"]
        
        self.x = signals.copy()
        
        ## Entry Signal
        if entryType == "enter":
            cond = (signals["LB_Delta"]<=0) & (signals["LB_Delta"].shift(1)>0)
            signals.loc[cond, "Signal"] = 1
            
        elif entryType == "exit":
            cond = (signals["LB_Delta"]>=0) & (signals["LB_Delta"].shift(1)<0)
            signals.loc[cond,"Signal"] = 1         
        
        ## Exit Signal
        if exitType == "enter":
            cond = (signals["UB_Delta"]>=0) & (signals["UB_Delta"].shift(1)<0)
            signals.loc[cond, "Signal"] = 0
        elif exitType == "exit":
            cond = (signals["UB_Delta"]<=0) & (signals["UB_Delta"].shift(1)>0)
            signals.loc[cond, "Signal"] = 0
            
            
        signals["Signal"] = signals["Signal"].ffill()
        ## If the indicator is Contra
        if self.sign == -1:
            signals["Signal"] = signals["Signal"].replace({1:0, 0:1})
        signals["Signal"] = signals["Signal"].fillna(0)
        
        ## Asset Weight for asset classes
        condition = signals["Signal"] == 1
        signals["BaseAsset"] = np.where(condition, 1, 0)
        signals["SecondAsset"] = np.where(condition, 0, 1)
        signals['Position'] = signals['Signal'].diff()
        
        return signals
    
    def calculate_total_return(self, signals, ma, thresh, entryType, exitType):
        
        ## Filter the data for required asset classes
        assetData = self.assetData.filter(items = [self.baseAsset, self.secondAsset])
        assetData = assetData.loc[min(signals.index):]
        
        ## MErge the price data and Singal
        data = pd.merge_asof(assetData, signals, left_index = True, right_index = True)

        price = data.filter(items = [self.baseAsset, self.secondAsset])

        sig = data.filter(items = ["BaseAsset", "SecondAsset"])
        sig.columns = [self.baseAsset, self.secondAsset]
        
        ## Percentage returns and Portfolio performance
        assetRtrn = price.pct_change()
        assetRtrn['Portfolio'] = (assetRtrn*sig.shift(1)).sum(axis = 1)
        
        ## Calculate the NAV
        port = assetRtrn.add(1).cumprod().fillna(1)*100

        ## CAGR
        dur = (port.index[-1]-port.index[0]).days/365
        port_cagr = (port['Portfolio'][-1]/port['Portfolio'][0])**(1/dur)-1
        
        ## Store the NAV in master dataframe
        strategy_nav = port["Portfolio"].copy()
        strategy_nav = strategy_nav.rename(f"{self.name}_MA({ma})_Thresh({thresh})_Entry({entryType})_Exit({exitType})")
        self.NAV = pd.concat([self.NAV, strategy_nav], axis = 1)
        
        return port_cagr
    
    def generateSignalReport(self,):
        
        if len(self.results) <= 0:
            print("There are no good results as per given input values. May be try with different values.")
            return
        
        ## Color map for signal
        color_map = {1 : 'green', 0 : 'red'}
        
        ## Create the PDF object
        signalPDF = PdfPages(f'{self.name}_Signal_Report_BollingerBand_{date.today()}.pdf')

        #######################
        ## Looping for Top results
        for idx, row in self.results.head(5).iterrows():
            
            ## Strategy Name
            strategyName = f'{self.name}_MA({row["MA"]})_Thresh({row["Threshold"]})_Entry({row["EntryType"]})_Exit({row["ExitType"]})'

            ## Plot Object
            fig, ax = plt.subplots(figsize=(20,10), dpi = 300)
            
            ## Generate Background color as Green | Red for Buy | Sell respectively.
            for zn in self.maSignal[strategyName].dropna().unique():
                ax.fill_between(self.maSignal[strategyName].reset_index()["index"],
                                0,1.5,
                                where = self.maSignal[strategyName] == zn,
                                alpha = .4,
                                color = color_map[zn],
                                label = zn, interpolate = True,
                                transform = ax.get_xaxis_transform())
            plt.yticks([])
            ## Plot the Largecap Data
            plt.plot(self.assetData["Largecap"], label = 'Nifty')
            plt.legend()
    
            ## CAGR | Rebalance Title Text.
            cagr= f"{round(row['Total_Return']*100,2)}-CAGR"
            rebal = f"{row['Num_Rebalances']}-Switches"   
            ## Title
            plt.suptitle(f"{self.name} -  {cagr} | {rebal}", fontsize = 20)
            plt.title(f"Method: Bollinger Band  |  MA: {row['MA']} |  Threshold: {row['Threshold']}  | Entry: {row['EntryType']} | Exit: {row['ExitType']}")
            
            ## Legend
            plt.legend(loc = "lower left", fontsize = 15)
            plt.tight_layout()
            
            ## Save the fig to PDF
            signalPDF.savefig(fig)
            ## Close the close
            plt.close()
            
        ## Close the PDF file
        signalPDF.close()

In [5]:
class RankingBacktester:
    def __init__(self, data, assetData, 
                 thresholdList = [0.3, 0.4, 0.5, 0.6], rankLookback = [52],
                 smootheningWindow = [1], 
                 sign = 1,minReturn = 0.12, maxRebalance = 1000,
                 baseAsset = "Largecap", secondAsset = "Gold", name = "MA"):
        ## Raw Data
        self.data = data
        
        ## MIN CAGR requirement and MAX rebalances
        self.minReturn = minReturn
        self.maxRebalance = maxRebalance
        
        ## List of all the Threshold
        self.thresholdList = thresholdList
        
        ## List of all Lookback periods
        self.rankLookback = rankLookback
        self.smootheningWindow = smootheningWindow
        ## Sign of the indicator
        self.sign = sign
        
        ## Asset Class NAV Data
        self.assetData = assetData
        ## Base Asset and Second Asset Name
        self.baseAsset = baseAsset
        self.secondAsset = secondAsset
        ## Strategy Name or Raw Data name
        self.name = name
        
        ## Empty Dataframe to store the results
        self.results = pd.DataFrame(columns=['Smooth', 'Threshold', 'Lookback', 'Total_Return', 'Num_Rebalances'])
        ## Empty Dataframe to store the NAV of each combos
        self.NAV = pd.DataFrame()
        ## Empty Dataframe to store the signal for each combos
        self.maSignal = pd.DataFrame()


    def run_backtest(self):
        
        ## Calculate the total number of iteration or number of combinations
        total_iterations = len(self.thresholdList) * len(self.rankLookback) * len(self.smootheningWindow)
        
        ## Progress BAR
        with tqdm(total=total_iterations, desc="Backtesting") as pbar:
            
            ## Loop for smoothening windows
            for smooth in self.smootheningWindow:
                ## Loop for Different Threshold Computation
                for threshold in self.thresholdList:
                    ## Loop for Different Lookback period
                    for lookback in self.rankLookback:

                        ## Function Call for signal generation
                        signals = self.generate_signals(smooth, threshold, lookback)
                        ## Function Call for return calculation
                        total_return = self.calculate_total_return(signals, smooth, threshold, lookback)
                        ## Calculate the number of signal switches
                        num_rebalances = len(signals[signals['Position'] != 0])

                        ## Store the result in a variable and append to master result dataframe
                        result = pd.DataFrame({'Smooth' : [smooth],
                                               'Threshold': [threshold],
                                               'Lookback': [lookback],
                                               'Total_Return': [total_return],
                                               'Num_Rebalances': [num_rebalances]})
                        self.results = pd.concat([self.results, result])

                        ## Store the signal for all combos in master dataframe
                        comboName = f"{self.name}_Smooth({smooth})_Thresh({threshold})_Lookback({lookback})"
                        self.maSignal = pd.concat([self.maSignal, 
                                                   signals["Signal"].rename(comboName)], axis = 1)
                        self.maSignal.index = pd.to_datetime(self.maSignal.index)

                        ## Update the Progress BAR  
                        pbar.update(1)
                    
        ## Filter the final result as per Min CAGR Requirements and MAX Rebalances.
        cond1 = self.results["Num_Rebalances"]<=self.maxRebalance
        cond2 = self.results["Total_Return"] >= self.minReturn
        self.results = self.results[cond1 & cond2]
        self.results = self.results.sort_values("Total_Return", ascending = False).reset_index(drop = True)
        
        # Function call to generate Signal charts
        self.generateSignalReport()
                                                       

    def generate_signals(self, smooth, threshold, lookback):
        
        ## Empty Dataframe to store the signal
        signals = pd.DataFrame(index=self.data.index)
        signals["Close"] = self.data["Close"].copy()
        signals["Smooth"] = signals["Close"].rolling(window = smooth).mean()

        ## Rolling Ranking
        flag = True if self.sign == 1 else False
        signals['Score'] = signals['Smooth'].rolling(window = lookback, min_periods=1).rank(pct = True, ascending = flag)
        
        ## Signal Condition
        signals['Signal'] = 0.0
        
        condition = signals["Score"] >= threshold
        ## Generate the Signal Columns
        signals['Signal'] = np.where(condition, 1, 0)
        
        ## Asset Weight for asset classes
        signals["BaseAsset"] = np.where(condition, 1, 0)
        signals["SecondAsset"] = np.where(condition, 0, 1)
        signals['Position'] = signals['Signal'].diff()
        
        return signals
    
    def calculate_total_return(self, signals, smooth, threshold, lookback):
        
        ## Filter the data for required asset classes
        assetData = self.assetData.filter(items = [self.baseAsset, self.secondAsset])
        assetData = assetData.loc[min(signals.index):]
        
        ## MErge the price data and Singal
        data = pd.merge_asof(assetData, signals, left_index = True, right_index = True)

        price = data.filter(items = [self.baseAsset, self.secondAsset])

        sig = data.filter(items = ["BaseAsset", "SecondAsset"])
        sig.columns = [self.baseAsset, self.secondAsset]
        
        ## Percentage returns and Portfolio performance
        assetRtrn = price.pct_change()
        assetRtrn['Portfolio'] = (assetRtrn*sig.shift(1)).sum(axis = 1)
        
        ## Calculate the NAV
        port = assetRtrn.add(1).cumprod().fillna(1)*100

        ## CAGR
        dur = (port.index[-1]-port.index[0]).days/365
        port_cagr = (port['Portfolio'][-1]/port['Portfolio'][0])**(1/dur)-1
        
        ## Store the NAV in master dataframe
        strategy_nav = port["Portfolio"].copy()
        strategy_nav = strategy_nav.rename(f"{self.name}_Smooth({smooth})_Thresh({threshold})_Lookback({lookback})")
        self.NAV = pd.concat([self.NAV, strategy_nav], axis = 1)
        
        return port_cagr
    
    def generateSignalReport(self,):
        
        if len(self.results) <= 0:
            print("There are no good results as per given input values. May be try with different values.")
            return
        
        ## Color map for signal
        color_map = {1 : 'green', 0 : 'red'}
        
        ## Create the PDF object
        signalPDF = PdfPages(f'{self.name}_Signal_Report_Ranking_{date.today()}.pdf')

        #######################
        ## Looping for Top results
        for idx, row in self.results.head(5).iterrows():
            
            ## Strategy Name
            strategyName = f'{self.name}_Smooth({row["Smooth"]})_Thresh({row["Threshold"]})_Lookback({row["Lookback"]})'

            ## Plot Object
            fig, ax = plt.subplots(figsize=(20,10), dpi = 300)
            
            ## Generate Background color as Green | Red for Buy | Sell respectively.
            for zn in self.maSignal[strategyName].dropna().unique():
                ax.fill_between(self.maSignal[strategyName].reset_index()["index"],
                                0,1.5,
                                where = self.maSignal[strategyName] == zn,
                                alpha = .4,
                                color = color_map[zn],
                                label = zn, interpolate = True,
                                transform = ax.get_xaxis_transform())
            plt.yticks([])
            ## Plot the Largecap Data
            plt.plot(self.assetData["Largecap"], label = 'Nifty')
            plt.legend()
    
            ## CAGR | Rebalance Title Text.
            cagr= f"{round(row['Total_Return']*100,2)}-CAGR"
            rebal = f"{row['Num_Rebalances']}-Switches"   
            ## Title
            plt.suptitle(f"{self.name} -  {cagr} | {rebal}", fontsize = 20)
            plt.title(f"Method: Ranking | Smoothening Window: {row['Smooth']}  |  Threshold: {row['Threshold']} | Lookback: {row['Lookback']}")
            
            ## Legend
            plt.legend(loc = "lower left", fontsize = 15)
            plt.tight_layout()
            
            ## Save the fig to PDF
            signalPDF.savefig(fig)
            ## Close the close
            plt.close()
            
        ## Close the PDF file
        signalPDF.close()

In [6]:
class ChannelBreakoutBacktester:
    def __init__(self, data, assetData, 
                 minWindow, maxWindow, smootheningWindow = [1],
                 sign = 1,minReturn = 0.12, maxRebalance = 1000,
                 baseAsset = "Largecap", secondAsset = "Gold", name = "MA"):
        ## Raw Data
        self.data = data
        self.minWindow = minWindow
        self.maxWindow = maxWindow
        self.smootheningWindow = smootheningWindow
        
        
        ## MIN CAGR requirement and MAX rebalances
        self.minReturn = minReturn
        self.maxRebalance = maxRebalance
    
        ## Sign of the indicator
        self.sign = sign
        
        ## Asset Class NAV Data
        self.assetData = assetData
        ## Base Asset and Second Asset Name
        self.baseAsset = baseAsset
        self.secondAsset = secondAsset
        ## Strategy Name or Raw Data name
        self.name = name
        
        ## Empty Dataframe to store the results
        self.results = pd.DataFrame(columns=['Lookback', "Smooth", 'Total_Return', 'Num_Rebalances'])
        ## Empty Dataframe to store the NAV of each combos
        self.NAV = pd.DataFrame()
        ## Empty Dataframe to store the signal for each combos
        self.maSignal = pd.DataFrame()
        
    def run_backtest(self):
        
        ## Calculate the total number of iteration or number of combinations
        total_iterations = len(range(self.minWindow, self.maxWindow+1)) * len(self.smootheningWindow)
        
        ## Progress BAR
        with tqdm(total=total_iterations, desc="Backtesting") as pbar:
            
            ## Loop for lookaback windows
            for window in range(self.minWindow, self.maxWindow+1):
                
                ## Loop for Different Smoothening period
                for smooth in self.smootheningWindow:
                
                    ## Function call to generate the signals
                    signals = self.generate_signals(window, smooth)

                    ## Function Call for return calculation
                    total_return = self.calculate_total_return(signals, window, smooth)

                    ## Calculate the number of signal switches
                    num_rebalances = len(signals[signals['Position'] != 0])

                    ## Store the result in a variable and append to master result dataframe
                    result = pd.DataFrame({'Lookback': [window],
                                           'Smooth' : [smooth],
                                           'Total_Return': [total_return],
                                           'Num_Rebalances': [num_rebalances]})
                    self.results = pd.concat([self.results, result])

                    ## Store the signal for all combos in master dataframe
                    comboName = f"{self.name}_Lookback({window})_Smooth({smooth})"
                    self.maSignal = pd.concat([self.maSignal, 
                                               signals["Signal"].rename(comboName)], axis = 1)
                    self.maSignal.index = pd.to_datetime(self.maSignal.index)

                    ## Update the Progress BAR  
                    pbar.update(1)

                
            ## Filter the final result as per Min CAGR Requirements and MAX Rebalances.
            cond1 = self.results["Num_Rebalances"]<=self.maxRebalance
            cond2 = self.results["Total_Return"] >= self.minReturn
            self.results = self.results[cond1 & cond2]
            self.results = self.results.sort_values("Total_Return", ascending = False).reset_index(drop = True)
        
            ## Function call to generate Signal charts
            self.generateSignalReport()
                
    def generate_signals(self, window, smooth):
        ## Empty Dataframe to store the signal
        signals = self.data.copy()
        signals["Close"] = signals["Close"].rolling(window = smooth).mean()
        
        ## Rolling Ranking
        flag = True if self.sign == 1 else False
        signals['MAX'] = signals['Close'].rolling(window = window).max().shift(1)
        signals['MIN'] = signals['Close'].rolling(window = window).min().shift(1)
        
        ## Signal Condition
        if self.sign == 1:
            buy_cond = (signals["Close"] >= signals["MAX"])
            sell_cond = (signals["Close"] < signals["MIN"])
        elif self.sign == -1:
            buy_cond = (signals["Close"] <= signals["MIN"]) 
            sell_cond = (signals["Close"] > signals["MAX"])
            
        
        ## Generate the Signal Columns
        signals.loc[buy_cond, 'Signal'] = 1
        signals.loc[sell_cond, 'Signal'] = 0
        signals["Signal"] = signals["Signal"].ffill()
        signals["Signal"] = signals["Signal"].fillna(0)
        
        ## Asset Weight for asset classes
        signals["BaseAsset"] = np.where(signals["Signal"] == 1, 1, 0)
        signals["SecondAsset"] = np.where(signals["Signal"] == 1, 0, 1)
        signals['Position'] = signals['Signal'].diff()
        
        return signals
    
    def calculate_total_return(self, signals, window, smooth):
        
        ## Filter the data for required asset classes
        assetData = self.assetData.filter(items = [self.baseAsset, self.secondAsset])
        assetData = assetData.loc[min(signals.index):]
        
        ## MErge the price data and Singal
        data = pd.merge_asof(assetData, signals, left_index = True, right_index = True)

        price = data.filter(items = [self.baseAsset, self.secondAsset])

        sig = data.filter(items = ["BaseAsset", "SecondAsset"])
        sig.columns = [self.baseAsset, self.secondAsset]
        
        ## Percentage returns and Portfolio performance
        assetRtrn = price.pct_change()
        assetRtrn['Portfolio'] = (assetRtrn*sig.shift(1)).sum(axis = 1)
        
        ## Calculate the NAV
        port = assetRtrn.add(1).cumprod().fillna(1)*100

        ## CAGR
        dur = (port.index[-1]-port.index[0]).days/365
        port_cagr = (port['Portfolio'][-1]/port['Portfolio'][0])**(1/dur)-1
        
        ## Store the NAV in master dataframe
        strategy_nav = port["Portfolio"].copy()
        strategy_nav = strategy_nav.rename(f"{self.name}_Lookback({window})_Smooth({smooth})")
        self.NAV = pd.concat([self.NAV, strategy_nav], axis = 1)
        
        return port_cagr
    
    def generateSignalReport(self,):
        
        if len(self.results) <= 0:
            print("There are no good results as per given input values. May be try with different values.")
            return
        
        ## Color map for signal
        color_map = {1 : 'green', 0 : 'red'}
        
        ## Create the PDF object
        signalPDF = PdfPages(f'{self.name}_Signal_Report_Channel-Breakout_{date.today()}.pdf')

        #######################
        ## Looping for Top results
        for idx, row in self.results.head(5).iterrows():
            
            ## Strategy Name
            strategyName = f'{self.name}_Lookback({row["Lookback"]})_Smooth({row["Smooth"]})'

            ## Plot Object
            fig, ax = plt.subplots(figsize=(20,10), dpi = 300)
            
            ## Generate Background color as Green | Red for Buy | Sell respectively.
            for zn in self.maSignal[strategyName].dropna().unique():
                ax.fill_between(self.maSignal[strategyName].reset_index()["index"],
                                0,1.5,
                                where = self.maSignal[strategyName] == zn,
                                alpha = .4,
                                color = color_map[zn],
                                label = zn, interpolate = True,
                                transform = ax.get_xaxis_transform())
            plt.yticks([])
            ## Plot the Largecap Data
            plt.plot(self.assetData["Largecap"], label = 'Nifty')
            plt.legend()
    
            ## CAGR | Rebalance Title Text.
            cagr= f"{round(row['Total_Return']*100,2)}-CAGR"
            rebal = f"{row['Num_Rebalances']}-Switches"   
            ## Title
            plt.suptitle(f"{self.name} -  {cagr} | {rebal}", fontsize = 20)
            plt.title(f"Method: Channel-Breakout | Lookback: {row['Lookback']} | Smooth: {row['Smooth']}")
            
            ## Legend
            plt.legend(loc = "lower left", fontsize = 15)
            plt.tight_layout()
            
            ## Save the fig to PDF
            signalPDF.savefig(fig)
            ## Close the close
            plt.close()
            
        ## Close the PDF file
        signalPDF.close()

In [7]:
class ChannelNormalizationBacktester:
    def __init__(self, data, assetData,
                 lookback, displacement, smootheningWindow
                 , sign = 1, minReturn = 0.12, maxRebalance = 1000,
                 baseAsset = "Largecap", secondAsset = "Gold", name = "MA"):
        ## Raw Data
        self.data = data
        ## Initialising Lookback period, Displacement values, Smoothening Window
        self.lookback = lookback
        self.displacement = displacement
        self.smootheningWindow = smootheningWindow
 
        ## MIN CAGR requirement and MAX rebalances
        self.minReturn = minReturn
        self.maxRebalance = maxRebalance
        
        ## Asset Class NAV Data
        self.assetData = assetData
        self.sign = sign
        
        ## Base Asset and Second Asset Name
        self.baseAsset = baseAsset
        self.secondAsset = secondAsset
        ## Strategy Name or Raw Data name
        self.name = name
        
        ## Empty Dataframe to store the results
        self.results = pd.DataFrame(columns=['Lookback', 'Displacement',"Smooth", "EntryType", "ExitType",
                                             'Total_Return', 'Num_Rebalances'])
        ## Empty Dataframe to store the NAV of each combos
        self.NAV = pd.DataFrame()
        ## Empty Dataframe to store the signal for each combos
        self.maSignal = pd.DataFrame()
        
        
    def run_backtest(self):

        total_iterations = 16*len(self.lookback)*len(self.displacement)*len(self.smootheningWindow)
        
        ## Progress BAR
        with tqdm(total=total_iterations, desc="Backtesting") as pbar:
            
            ## Looping for two type of entry signal
            for entryType in ["LBenter", "LBexit", "UBenter", "UBexit"]:
                ## Looping for two type of exit signal
                for exitType in ["UBenter", "UBexit", "LBenter", "LBexit"]:
                    ## Looping for multiple MA values
                    for lookback in self.lookback:
                        ## Looping for multiple displacement values
                        for displacement in self.displacement:
                            ## Looping for smoothening period
                            for smooth in self.smootheningWindow:
                                                        
                                ## Function Call for signal generation
                                signals = self.generate_signals(lookback, displacement, smooth, entryType,exitType)
                                ## Function Call for return calculation
                                total_return = self.calculate_total_return(signals,lookback,displacement,smooth, entryType,exitType)
                                ## Calculate the number of signal switches
                                num_rebalances = len(signals[signals['Position'] != 0])
                                
                                ## Store the result in a variable and append to master result dataframe
                                result = pd.DataFrame({'Lookback': [lookback],
                                                       'Displacement': [50-displacement],
                                                       'Smooth' : [smooth],
                                                       "EntryType" : [entryType],
                                                       "ExitType" : [exitType],
                                                       'Total_Return': [total_return],
                                                       'Num_Rebalances': [num_rebalances]})
                                self.results = pd.concat([self.results, result])
                                
                                                                      
                                ## Store the signal for all combos in master dataframe
                                comboName = f"Lookback({lookback})_Displacement({50-displacement})_Smooth({smooth})_Entry({entryType})_Exit({exitType})"
                                self.maSignal = pd.concat([self.maSignal, 
                                                           signals["Signal"].rename(comboName)], axis = 1)
                                self.maSignal.index = pd.to_datetime(self.maSignal.index)

                                pbar.update(1)
                            
                            
            ## Filter the final result as per Min CAGR Requirements and MAX Rebalances.
            cond1 = self.results["Num_Rebalances"]<=self.maxRebalance
            cond2 = self.results["Total_Return"] >= self.minReturn
            self.results = self.results[cond1 & cond2]
            self.results = self.results.sort_values("Total_Return", ascending = False).reset_index(drop = True)

            ## Function call to generate Signal charts
            self.generateSignalReport()

                      
    def generate_signals(self, lookback, displacement,smooth, entryType, exitType):
        
        ## Empty Dataframe to store the signal
        signals = pd.DataFrame(index=self.data.index)
        ## Store the indcator values
        signals["Value"] = self.data["Close"].copy()
        ## Smoothen the indicator values
        signals["Smooth"] = signals["Value"].rolling(window = smooth).mean()
        
        ## Convert the indicator to Channel Normalized operator
        def cno(series):
            return ((series - min(series))/(max(series) - min(series)))[-1]
        signals["CNO"] = signals["Smooth"].rolling(window = lookback).apply(cno)*100
        
        ## Defining the lower and upper band
        signals["UB"] = 50 + displacement
        signals["LB"] = 50 - displacement
    
        ## Oscillator of LB and UB
        signals["LB_Delta"] = signals["CNO"] - signals["LB"]
        signals["UB_Delta"] = signals["CNO"] - signals["UB"]
        
        ## Entry Signal: Enter the Lower Band
        if entryType == "LBenter":
            cond = (signals["LB_Delta"]<=0) & (signals["LB_Delta"].shift(1)>0)
            signals.loc[cond, "Signal"] = 1
        ## Exit the Lower Band
        elif entryType == "LBexit":
            cond = (signals["LB_Delta"]>=0) & (signals["LB_Delta"].shift(1)<0)
            signals.loc[cond, "Signal"] = 1
        ## Enter the Upper Band
        elif entryType == "UBenter":
            cond = (signals["UB_Delta"]>=0) & (signals["UB_Delta"].shift(1)<0)
            signals.loc[cond,"Signal"] = 1
        ## Exit the Upper Band
        elif entryType == "UBexit":
            cond = (signals["UB_Delta"]<=0) & (signals["UB_Delta"].shift(1)>0)
            signals.loc[cond,"Signal"] = 1         
        
        ## Exit Signal: Enter the Upper Band
        if exitType == "UBenter":
            cond = (signals["UB_Delta"]>=0) & (signals["UB_Delta"].shift(1)<0)
            signals.loc[cond, "Signal"] = 0
        ## Exit the Upper Band
        elif exitType == "UBexit":
            cond = (signals["UB_Delta"]<=0) & (signals["UB_Delta"].shift(1)>0)
            signals.loc[cond, "Signal"] = 0
        elif exitType == "LBenter":
            cond = (signals["LB_Delta"]<=0) & (signals["LB_Delta"].shift(1)>0)
            signals.loc[cond, "Signal"] = 0
        elif exitType == "LBexit":
            cond = (signals["LB_Delta"]>=0) & (signals["LB_Delta"].shift(1)<0)
            signals.loc[cond, "Signal"] = 0
        signals["Signal"] = signals["Signal"].ffill()
        
#         ## If the indicator is Contra
#         if self.sign == -1:
#             signals["Signal"] = signals["Signal"].replace({1:0, 0:1})
        signals["Signal"] = signals["Signal"].fillna(0)
        
        ## Asset Weight for asset classes
        condition = signals["Signal"] == 1
        signals["BaseAsset"] = np.where(condition, 1, 0)
        signals["SecondAsset"] = np.where(condition, 0, 1)
        signals['Position'] = signals['Signal'].diff()
        
        return signals
    
    def calculate_total_return(self, signals, lookback, displacement, smooth, entryType, exitType):
        
        ## Filter the data for required asset classes
        assetData = self.assetData.filter(items = [self.baseAsset, self.secondAsset])
        assetData = assetData.loc[min(signals.index):]
        
        ## MErge the price data and Singal
        data = pd.merge_asof(assetData, signals, left_index = True, right_index = True)

        price = data.filter(items = [self.baseAsset, self.secondAsset])

        sig = data.filter(items = ["BaseAsset", "SecondAsset"])
        sig.columns = [self.baseAsset, self.secondAsset]
        
        ## Percentage returns and Portfolio performance
        assetRtrn = price.pct_change()
        assetRtrn['Portfolio'] = (assetRtrn*sig.shift(1)).sum(axis = 1)
        
        ## Calculate the NAV
        port = assetRtrn.add(1).cumprod().fillna(1)*100

        ## CAGR
        dur = (port.index[-1]-port.index[0]).days/365
        port_cagr = (port['Portfolio'][-1]/port['Portfolio'][0])**(1/dur)-1
        
        ## Store the NAV in master dataframe
        strategy_nav = port["Portfolio"].copy()
        strategy_nav = strategy_nav.rename(f"{self.name}_Lookback({lookback})_Displacement({50-displacement})_Smooth({smooth})_Entry({entryType})_Exit({exitType})")
        
        self.NAV = pd.concat([self.NAV, strategy_nav], axis = 1)
        
        return port_cagr
    
    def generateSignalReport(self,):
        
        if len(self.results) <= 0:
            print("There are no good results as per given input values. May be try with different values.")
            return
        
        ## Color map for signal
        color_map = {1 : 'green', 0 : 'red'}
        
        ## Create the PDF object
        signalPDF = PdfPages(f'{self.name}_Signal_Report_Channel Normalization Operator_{date.today()}.pdf')

        #######################
        ## Looping for Top results
        for idx, row in self.results.head(5).iterrows():
            
            ## Strategy Name
            strategyName = f'Lookback({row["Lookback"]})_Displacement({row["Displacement"]})_Smooth({row["Smooth"]})_Entry({row["EntryType"]})_Exit({row["ExitType"]})'
            
            ## Plot Object
            fig, ax = plt.subplots(figsize=(20,10), dpi = 300)
            
            ## Generate Background color as Green | Red for Buy | Sell respectively.
            for zn in self.maSignal[strategyName].dropna().unique():
                ax.fill_between(self.maSignal[strategyName].reset_index()["index"],
                                0,1.5,
                                where = self.maSignal[strategyName] == zn,
                                alpha = .4,
                                color = color_map[zn],
                                label = zn, interpolate = True,
                                transform = ax.get_xaxis_transform())
            plt.yticks([])
            ## Plot the Largecap Data
            plt.plot(self.assetData["Largecap"], label = 'Nifty')
            plt.legend()
    
            ## CAGR | Rebalance Title Text.
            cagr= f"{round(row['Total_Return']*100,2)}-CAGR"
            rebal = f"{row['Num_Rebalances']}-Switches"   
            ## Title
            plt.suptitle(f"{self.name} -  {cagr} | {rebal}", fontsize = 20)
            plt.title(f"Method: Channel Normalization Operator  |  Lookback: {row['Lookback']} |  Displacement: {row['Displacement']} | Smooth: {row['Smooth']}| Entry: {row['EntryType']} | Exit: {row['ExitType']}")
            
            ## Legend
            plt.legend(loc = "lower left", fontsize = 15)
            plt.tight_layout()
            
            ## Save the fig to PDF
            signalPDF.savefig(fig)
            ## Close the close
            plt.close()
            
        ## Close the PDF file
        signalPDF.close()

In [11]:
assetData

,Largecap,Junior,Midcap,Smallcap,S&P500,Nasdaq 100,Gold,GovtBond,CorpBond,Liquid,Cash
Date,,,,,,,,,,,
2006-07-03,32.02203,5.276884,13.966204,3.602988,3.274646,7.030136,9.359685,7.015935,287.906603,1145.172263,1
2006-07-04,31.89682,5.266227,13.930497,3.588346,3.274646,7.030136,9.317014,7.109784,287.954640,1145.323383,1
2006-07-05,32.48864,5.337272,14.079867,3.609277,3.250909,6.886432,9.361452,6.760859,288.002677,1145.500921,1
2006-07-06,32.09369,5.314057,14.007215,3.592867,3.263985,6.888667,9.421845,6.754732,288.052635,1145.652759,1
2006-07-07,31.28193,5.185127,13.731985,3.539116,3.231380,6.787202,9.487632,6.748314,288.100671,1145.808154,1
...,...,...,...,...,...,...,...,...,...,...,...
2024-01-24,237.29000,56.180000,179.410000,31.276900,18.156800,141.570000,52.910000,24.990000,1178.590000,2564.371000,1
2024-01-25,236.15000,56.210000,178.490000,31.443300,18.248700,141.890000,52.720000,25.010000,1181.140000,2564.797500,1
2024-01-29,240.05000,56.940000,181.100000,31.846200,18.381200,141.650000,52.970000,25.000000,1180.900000,2565.229700,1


In [8]:
assetData = pd.read_excel("./AssetClassNAV.xlsx", sheet_name = "AssetData", index_col = 0)

# data = pd.read_excel("nifty_gold_raw_price.xlsx", index_col = 0).set_index("Date")
data = yf.download("^NSEI")[["Close"]]

# master = pd.read_excel("./MADP-PACKAGE/MADP-Master-Data.xlsx", index_col = 0)

# master = pd.read_excel("./onlyCol.xlsx", index_col = 0)

# target = "Gold_to_Copper"
# data = master[[target]].rename(columns = {target : "Close"}).resample("W-Fri").last()

[*********************100%%**********************]  1 of 1 completed


In [22]:
####################
## Moving Average ##
####################

# Example usage:
# Assuming 'data' is your historical price data in a DataFrame with a 'Close' column
min_fast_ma = 10        ## Minimum Value of Fast Moving Average
max_fast_ma = 20        ## Maximum Value of Fast Moving Average
min_slow_ma = 20        ## Minimum Value of Slow Moving Average
max_slow_ma = 30        ## Maximum Value of Slow Moving Avergae
minReturn = .01          ## Minimum Return that is expected from the reuslts of Indicator
maxRebalance = 1000     ## Maximum Rebalance / Switches an indicator can have.
sign = -1               ## +1 Value means Higher value is desired for Base Asset, vice versa for -1
maType = "ema"       ## Type of MA to be used, either "simple" or "ema"
baseAsset = "Largecap"  ## Base Asssset Class
secondAsset = "Gold"    ## Second Asset Class
name = "Nifty SMA"      ## Name of the Indicator

backtester = MovingAverageBacktester(data = data,
                                     min_fast_ma = min_fast_ma, 
                                     max_fast_ma = max_fast_ma, 
                                     min_slow_ma = min_slow_ma, 
                                     max_slow_ma = max_slow_ma,
                                     assetData = assetData, 
                                     sign = sign,
                                     maType = maType,
                                     minReturn = minReturn, 
                                     maxRebalance = maxRebalance,
                                     baseAsset = baseAsset,
                                     secondAsset = secondAsset,
                                     name = name)
backtester.run_backtest()
result = backtester.results

Backtesting: 100%|████████████████████████████| 121/121 [00:01<00:00, 92.37it/s]


In [24]:
####################
## Bollinger Band ##
####################

# Example usage:
# Assuming 'data' is your historical price data in a DataFrame with a 'Close' column
min_ma = 12             ## Minimum Lookback period of Bollinger Band
max_ma = 12             ## Maximum Lookback periods of Bollinger Band
min_thresh = 1          ## Minimum threshold for Band width
max_thresh = 2          ## Maximum threhsold for Band width
minReturn = .0          ## Minimum Return that is expected from the reuslts of Indicator
maxRebalance = 100      ## Maximum Rebalance / Switches an indicator can have.
sign = 1                ## +1 Value means Higher value is desired for Base Asset, vice versa for -1
baseAsset = "Laregcap"  ## Base Asssset Class
secondAsset = "Gold"    ## Second Asset Class
name = "Nifty BB"       ## Name of the Indicator

backtester = BollingerBandBacktester(data = data,
                                     min_ma = min_ma, 
                                     max_ma = max_ma, 
                                     min_thresh = min_thresh, 
                                     max_thresh = max_thresh,
                                     assetData = assetData,
                                     sign = sign,
                                     minReturn = minReturn, 
                                     maxRebalance = maxRebalance,
                                     baseAsset = baseAsset,
                                     secondAsset = secondAsset,
                                     name = name)
backtester.run_backtest()
result = backtester.results

Backtesting: 100%|██████████████████████████████| 44/44 [00:00<00:00, 81.04it/s]


In [25]:
#############
## Ranking ##
#############

# Example usage:
# Assuming 'data' is your historical price data in a DataFrame with a 'Close' column
minReturn = .10
maxRebalance = 100
thresholdList = [0.3,0.4,0.5,0.6]
rankLookback = [26, 52, 78, 104]
smootheningWindow = [1,2,3,4,5]
sign = 1
baseAsset = "Largecap"
secondAsset = "Gold"
name = "Nifty Rank"
backtester = RankingBacktester(data = data,
                               assetData = assetData, 
                             thresholdList = thresholdList,
                             rankLookback = rankLookback,
                             smootheningWindow = smootheningWindow,
                             sign = sign,
                             minReturn = minReturn, 
                             maxRebalance = maxRebalance,
                             baseAsset = baseAsset,
                             secondAsset = secondAsset,
                             name = name)
backtester.run_backtest()
result = backtester.results

Backtesting: 100%|██████████████████████████████| 80/80 [00:00<00:00, 84.38it/s]


In [26]:
######################
## Channel Breakout ##
######################

# Example usage:
# Assuming 'data' is your historical price data in a DataFrame with a 'Close' column
minWindow = 170
maxWindow = 180
smootheningWindow = list(range(5,10))
minReturn = .12
maxRebalance = 50
sign = 1
baseAsset = "Largecap"
secondAsset = "Gold"
name = "Nifty"

data = yf.download("^NSEI", progress = False)[["Close"]].copy()
assetData = pd.read_excel("AssetClassNAV.xlsx", index_col = 0)

backtester = ChannelBreakoutBacktester(data = data,
                                       assetData = assetData, 
                                     minWindow = minWindow,
                                     maxWindow = maxWindow,
                                     smootheningWindow = smootheningWindow,
                                     sign = sign,  
                                     minReturn = minReturn, 
                                     maxRebalance = maxRebalance,
                                     baseAsset = baseAsset,
                                     secondAsset = secondAsset,
                                     name = name)
backtester.run_backtest()
result = backtester.results

Backtesting: 100%|██████████████████████████████| 55/55 [00:01<00:00, 53.40it/s]


In [31]:
###########################
## Channel Normalization ##
###########################

# Example usage:
# Assuming 'data' is your historical price data in a DataFrame with a 'Close' column
lookback = [100]#[50, 100, 150, 200, 252]
displacement = [10, 20, 30, 40]
smootheningWindow = [5]#[5, 10, 15]
minReturn = .10
maxRebalance = 50
sign = 1
baseAsset = "Largecap"
secondAsset = "Cash"
name = "Nifty"

backtester = ChannelNormalizationBacktester(data = data,
                                         assetData = assetData,
                                         lookback = lookback,
                                         displacement = displacement,
                                         smootheningWindow = smootheningWindow,
                                         sign = sign,
                                         minReturn = minReturn, 
                                         maxRebalance = maxRebalance,
                                         baseAsset = baseAsset,
                                         secondAsset = secondAsset,
                                         name = name)
backtester.run_backtest()
result = backtester.results

Backtesting: 100%|██████████████████████████████| 64/64 [00:15<00:00,  4.08it/s]
